In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# ================= CONFIGURATION =================
# Load the V2 model you just made
OLD_MODEL = "house_price_model_v2.h5" 
NEW_MODEL = "house_price_model_v3.h5" # We will save a better version
DATA_FILE = "cleaned_dataset.csv"
IMG_FOLDER = "data/house_images"
IMG_SIZE = 128
# =================================================

def load_data():
    print("--- 📂 Loading Data ---")
    df = pd.read_csv(DATA_FILE)
    
    # Filter: Only keep rows where image exists
    df['img_path'] = df['id'].apply(lambda x: os.path.join(IMG_FOLDER, f"image_{x}.jpg"))
    df = df[df['img_path'].apply(os.path.exists)]
    
    # 1. Process Images
    print(f"Loading {len(df)} images (this keeps the best ones)...")
    images = []
    for img_path in df['img_path']:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
    
    X_img = np.array(images) / 255.0
    
    # 2. Process Stats (Lat/Long/Bed/Bath/etc)
    feature_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'lat', 'long']
    scaler = MinMaxScaler()
    X_stats = scaler.fit_transform(df[feature_cols].values)
    
    # 3. Process Prices
    y = df['price'].values
    
    return X_img, X_stats, y

def main():
    # 1. Load Data
    X_img, X_stats, y = load_data()
    
    # Split Data
    Xi_train, Xi_test, Xs_train, Xs_test, y_train, y_test = train_test_split(
        X_img, X_stats, y, test_size=0.2, random_state=42
    )

    # 2. Load the Old Brain
    print(f"\n--- 🧠 Loading {OLD_MODEL} ---")
    model = tf.keras.models.load_model(OLD_MODEL, compile=False)

    # 3. COMPILING WITH LOW LEARNING RATE (The Secret Sauce)
    # We use 0.0001 instead of 0.001 to make it "Fine Tune"
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(
        optimizer=optimizer, 
        loss='mse', 
        metrics=['mae']
    )

    # 4. Train Gently
    print("\n--- 🎓 Fine-Tuning (Slow & Steady) ---")
    history = model.fit(
        [Xi_train, Xs_train], y_train,
        validation_data=([Xi_test, Xs_test], y_test),
        epochs=15,           # Short run
        batch_size=32
    )

    # 5. Save V3
    model.save(NEW_MODEL)
    print(f"\n✅ Saved smarter model to: {NEW_MODEL}")

    # Check improvement
    final_loss = history.history['val_loss'][-1]
    print(f"📉 Final Validation Error (MSE): {int(final_loss):,}")
    print(f"root mean squared error approx: ${int(np.sqrt(final_loss)):,}")

if __name__ == "__main__":
    main()

C:\Users\sumit\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


--- 📂 Loading Data ---
Loading 12892 images (this keeps the best ones)...

--- 🧠 Loading house_price_model_v2.h5 ---

--- 🎓 Fine-Tuning (Slow & Steady) ---
Epoch 1/15
323/323 ━━━━━━━━━━━━━━━━━━━━ 213s 641ms/step - loss: 65651998720.0000 - mae: 145976.3750 - val_loss: 57624096768.0000 - val_mae: 131627.4844
Epoch 2/15
323/323 ━━━━━━━━━━━━━━━━━━━━ 200s 618ms/step - loss: 63851315200.0000 - mae: 145229.0781 - val_loss: 56793411584.0000 - val_mae: 130552.7812
Epoch 3/15
323/323 ━━━━━━━━━━━━━━━━━━━━ 230s 712ms/step - loss: 62434127872.0000 - mae: 144839.7031 - val_loss: 58562113536.0000 - val_mae: 145488.0938
Epoch 4/15
323/323 ━━━━━━━━━━━━━━━━━━━━ 231s 717ms/step - loss: 61414768640.0000 - mae: 145242.8281 - val_loss: 57425211392.0000 - val_mae: 128790.7422
Epoch 5/15
323/323 ━━━━━━━━━━━━━━━━━━━━ 272s 843ms/step - loss: 60010278912.0000 - mae: 145204.2656 - val_loss: 57254133760.0000 - val_mae: 145947.4531
Epoch 6/15
323/323 ━━━━━━━━━━━━━━━━━━━━ 272s 843ms/step - loss: 61338443776.0000 - m


✅ Saved smarter model to: house_price_model_v3.h5
📉 Final Validation Error (MSE): 52,475,011,072
root mean squared error approx: $229,074
